# Setup 

### Import libraries

In [1]:
import yaml 
from pathlib import Path
import os
path = Path("C:/Users/HP/Desktop/UNOS data/codes/simulation-waitline-unos").resolve()  # Get the absolute path
os.chdir(path)
import prediction as pred 
from data import *
from simulation import * 
import copy

Load data from UNOS and dictionaries to map ethnicities and blood type compatibility

In [2]:
df_base = load_data()   
yaml_file_path = "C:/Users/HP/Desktop/UNOS data/codes/simulation-waitline-unos/mapping_data.yaml"
with open(yaml_file_path, "r") as file:
    loaded_data = yaml.safe_load(file)
ethcat_mapping = loaded_data["ethcat_mapping"]
blood_type_compatibility = loaded_data["blood_type_compatibility"]

Get proportions by gender from UNOS sample

In [3]:
real_gender_df = df_base.GENDER.value_counts(normalize=True).reset_index()
real_gender_df.columns = ['gender', 'sample_proportion']
real_gender_df

,gender,sample_proportion
0,M,0.64827
1,F,0.35173


Get proportions by ethnicity from UNOS sample

In [4]:
real_ethcat_df = df_base.ETHCAT.value_counts(normalize=True).reset_index()
real_ethcat_df.ETHCAT = real_ethcat_df.ETHCAT.map(ethcat_mapping)
real_ethcat_df.columns = ['ethcat', 'sample_proportion']
real_ethcat_df

,ethcat,sample_proportion
0,"White, Non-Hispanic",0.439416
1,"Black, Non-Hispanic",0.325258
2,Hispanic/Latino,0.159551
3,"Asian, Non-Hispanic",0.053213
4,"Amer Ind/Alaska Native, Non-Hispanic",0.013618
5,"Multiracial, Non-Hispanic",0.004764
6,"Native Hawaiian/Other Pacific Islander, Non-Hi...",0.004180


Load predictors and create events

In [5]:
decision_tree = pred.SurvivalPredictionModel(model_path='models\decision_tree_1212.pkl')
cox_prop = pred.SurvivalPredictionModel(model_path='models\coxprop_0912.pkl')
random_forest = pred.SurvivalPredictionModel(model_path=r'models\forest_0912.pkl')

replicates = 1
events_per_replication =create_list_of_events(n_events = 10000, replications=replicates)

events_per_replication_p1 = copy.deepcopy(events_per_replication)
events_per_replication_p2 = copy.deepcopy(events_per_replication)
events_per_replication_p3 = copy.deepcopy(events_per_replication)



<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\HP\AppData\Local\Temp\ipykernel_22620\3663900534.py:1: SyntaxWarning: invalid escape sequence '\d'
  decision_tree = pred.SurvivalPredictionModel(model_path='models\decision_tree_1212.pkl')
C:\Users\HP\AppData\Local\Temp\ipykernel_22620\3663900534.py:2: SyntaxWarning: invalid escape sequence '\c'
  cox_prop = pred.SurvivalPredictionModel(model_path='models\coxprop_0912.pkl')


models\decision_tree_1212.pkl
Model loaded successfully from models\decision_tree_1212.pkl.
models\coxprop_0912.pkl
Model loaded successfully from models\coxprop_0912.pkl.
models\forest_0912.pkl
Model loaded successfully from models\forest_0912.pkl.


# Survival Decision tree

## Policy 1

In [6]:
average_waiting_times, number_of_matches_list, results_p1_no, results_p1_ll, results_p1_fl = run_simulation(events_per_replication_p1,  replicates, decision_tree, policy='p1', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [04:19, 42.21time unit/s]                       


### recipients who receive a kidney

In [7]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p1_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [8]:

df_waiting_time_no.to_csv('results/decision_tree_p1_eth_no.csv') 
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1098.348,NaN,15.0,NaN,2907.285,NaN,0.011,0.014
1,"Asian, Non-Hispanic",1095.914,NaN,64.0,NaN,2761.018,NaN,0.046,0.053
2,"Black, Non-Hispanic",1091.468,NaN,448.0,NaN,2745.143,NaN,0.324,0.325
3,Hispanic/Latino,1091.086,NaN,247.0,NaN,2749.024,NaN,0.179,0.160
4,"Multiracial, Non-Hispanic",1107.965,NaN,9.0,NaN,3587.014,NaN,0.007,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1036.353,NaN,4.0,NaN,3095.543,NaN,0.003,0.004
6,"White, Non-Hispanic",1123.706,NaN,594.0,NaN,2708.821,NaN,0.430,0.439


In [9]:

df_gender_no.to_csv('results/decision_tree_p1_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1104.012288,NaN,504.0,NaN,2729.315952,NaN,0.364953,0.35173
1,M,1106.346316,NaN,877.0,NaN,2744.900017,NaN,0.635047,0.64827


### patients who leave the waitlist

In [10]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p1_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [11]:
df_waiting_time_ll.to_csv('results/decision_tree_p1_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",525.647,NaN,9.0,NaN,0.0,NaN,0.017,0.014
1,"Asian, Non-Hispanic",589.195,NaN,29.0,NaN,0.0,NaN,0.056,0.053
2,"Black, Non-Hispanic",509.565,NaN,170.0,NaN,0.0,NaN,0.326,0.325
3,Hispanic/Latino,490.133,NaN,92.0,NaN,0.0,NaN,0.176,0.160
4,"Multiracial, Non-Hispanic",238.575,NaN,1.0,NaN,0.0,NaN,0.002,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",700.181,NaN,2.0,NaN,0.0,NaN,0.004,0.004
6,"White, Non-Hispanic",532.598,NaN,219.0,NaN,0.0,NaN,0.420,0.439


In [12]:
df_gender_ll.to_csv('results/decision_tree_p1_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,528.310029,NaN,182.0,NaN,0.0,NaN,0.348659,0.35173
1,M,516.650988,NaN,340.0,NaN,0.0,NaN,0.651341,0.64827


### patients who stay in final list

In [13]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p1_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [14]:
df_waiting_time_fl.to_csv('results/decision_tree_p1_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",537.791,NaN,3.0,NaN,0.0,NaN,0.012,0.014
1,"Asian, Non-Hispanic",241.181,NaN,6.0,NaN,0.0,NaN,0.025,0.053
2,"Black, Non-Hispanic",538.785,NaN,98.0,NaN,0.0,NaN,0.405,0.325
3,Hispanic/Latino,587.825,NaN,43.0,NaN,0.0,NaN,0.178,0.160
4,"Multiracial, Non-Hispanic",247.441,NaN,2.0,NaN,0.0,NaN,0.008,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1064.119,NaN,1.0,NaN,0.0,NaN,0.004,0.004
6,"White, Non-Hispanic",566.884,NaN,89.0,NaN,0.0,NaN,0.368,0.439


In [15]:
df_gender_fl.to_csv('results/decision_tree_p1_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,531.673289,NaN,82.0,NaN,0.0,NaN,0.338843,0.35173
1,M,559.702167,NaN,160.0,NaN,0.0,NaN,0.661157,0.64827


In [16]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/decision_tree_p1_eth_prob.csv')

## Policy 2

In [17]:
average_waiting_times, number_of_matches_list, results_p2_no, results_p2_ll, results_p2_fl = run_simulation(events_per_replication_p2,  replicates, decision_tree, policy='p2', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [01:34, 116.42time unit/s]                       


### recipients who receive a kidney

In [18]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p2_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [19]:
df_waiting_time_no.to_csv('results/decision_tree_p2_eth_no.csv') 
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1008.641,NaN,16.0,NaN,2492.009,NaN,0.012,0.014
1,"Asian, Non-Hispanic",1080.577,NaN,62.0,NaN,2706.241,NaN,0.045,0.053
2,"Black, Non-Hispanic",1120.023,NaN,456.0,NaN,2541.930,NaN,0.330,0.325
3,Hispanic/Latino,1059.806,NaN,244.0,NaN,2549.400,NaN,0.177,0.160
4,"Multiracial, Non-Hispanic",1089.325,NaN,9.0,NaN,2212.539,NaN,0.007,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1024.543,NaN,6.0,NaN,2607.657,NaN,0.004,0.004
6,"White, Non-Hispanic",1106.316,NaN,588.0,NaN,2478.240,NaN,0.426,0.439


In [20]:
df_gender_no.to_csv('results/decision_tree_p2_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1118.686414,NaN,506.0,NaN,2508.400723,NaN,0.366401,0.35173
1,M,1088.990643,NaN,875.0,NaN,2528.395476,NaN,0.633599,0.64827


### patients who leave the waitlist

In [21]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p2_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [22]:
df_waiting_time_ll.to_csv('results/decision_tree_p2_eth_ll.csv') 
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",460.861,NaN,8.0,NaN,0.0,NaN,0.015,0.014
1,"Asian, Non-Hispanic",634.286,NaN,30.0,NaN,0.0,NaN,0.058,0.053
2,"Black, Non-Hispanic",523.739,NaN,166.0,NaN,0.0,NaN,0.321,0.325
3,Hispanic/Latino,525.302,NaN,93.0,NaN,0.0,NaN,0.180,0.160
4,"Multiracial, Non-Hispanic",238.575,NaN,1.0,NaN,0.0,NaN,0.002,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",301.240,NaN,1.0,NaN,0.0,NaN,0.002,0.004
6,"White, Non-Hispanic",575.954,NaN,218.0,NaN,0.0,NaN,0.422,0.439


In [23]:
df_gender_ll.to_csv('results/decision_tree_p2_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,538.881565,NaN,179.0,NaN,0.0,NaN,0.346228,0.35173
1,M,556.648640,NaN,338.0,NaN,0.0,NaN,0.653772,0.64827


### patients who stay in final list

In [24]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p2_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [25]:
df_waiting_time_fl.to_csv('results/decision_tree_p2_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",537.791,NaN,3.0,NaN,0.0,NaN,0.012,0.014
1,"Asian, Non-Hispanic",636.947,NaN,7.0,NaN,0.0,NaN,0.028,0.053
2,"Black, Non-Hispanic",547.189,NaN,95.0,NaN,0.0,NaN,0.386,0.325
3,Hispanic/Latino,675.587,NaN,45.0,NaN,0.0,NaN,0.183,0.160
4,"Multiracial, Non-Hispanic",247.441,NaN,2.0,NaN,0.0,NaN,0.008,0.005
5,"White, Non-Hispanic",670.977,NaN,94.0,NaN,0.0,NaN,0.382,0.439


In [26]:
df_gender_fl.to_csv('results/decision_tree_p2_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,562.906555,NaN,84.0,NaN,0.0,NaN,0.341463,0.35173
1,M,646.536487,NaN,162.0,NaN,0.0,NaN,0.658537,0.64827


In [27]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/decision_tree_p2_eth_prob.csv')

## Policy 3

In [28]:
average_waiting_times, number_of_matches_list, results_p3_no, results_p3_ll, results_p3_fl = run_simulation(events_per_replication_p3,  replicates, random_forest, policy='p3', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [21:44,  8.40time unit/s]                         


### recipients who receive a kidney

In [29]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p3_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [30]:
df_waiting_time_no.to_csv('results/decision_tree_p3_eth_no.csv')  
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1056.926,NaN,16.0,NaN,3014.436,NaN,0.012,0.014
1,"Asian, Non-Hispanic",1097.803,NaN,64.0,NaN,2957.480,NaN,0.046,0.053
2,"Black, Non-Hispanic",1130.518,NaN,454.0,NaN,2979.809,NaN,0.329,0.325
3,Hispanic/Latino,1066.911,NaN,242.0,NaN,2983.692,NaN,0.175,0.160
4,"Multiracial, Non-Hispanic",1152.263,NaN,9.0,NaN,2947.703,NaN,0.007,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1001.366,NaN,5.0,NaN,2897.080,NaN,0.004,0.004
6,"White, Non-Hispanic",1009.422,NaN,591.0,NaN,2965.052,NaN,0.428,0.439


In [31]:
df_gender_no.to_csv('results/decision_tree_p3_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1070.593836,NaN,510.0,NaN,2975.077897,NaN,0.369298,0.35173
1,M,1061.493236,NaN,871.0,NaN,2971.833510,NaN,0.630702,0.64827


### patients who leave the waitlist

In [32]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p3_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [33]:
df_waiting_time_ll.to_csv('results/decision_tree_p3_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",460.861,NaN,8.0,NaN,0.0,NaN,0.016,0.014
1,"Asian, Non-Hispanic",603.222,NaN,27.0,NaN,0.0,NaN,0.053,0.053
2,"Black, Non-Hispanic",543.445,NaN,170.0,NaN,0.0,NaN,0.332,0.325
3,Hispanic/Latino,574.308,NaN,96.0,NaN,0.0,NaN,0.188,0.160
4,"Multiracial, Non-Hispanic",238.575,NaN,1.0,NaN,0.0,NaN,0.002,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",700.181,NaN,2.0,NaN,0.0,NaN,0.004,0.004
6,"White, Non-Hispanic",570.249,NaN,208.0,NaN,0.0,NaN,0.406,0.439


In [34]:
df_gender_ll.to_csv('results/decision_tree_p3_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,561.186147,NaN,177.0,NaN,0.0,NaN,0.345703,0.35173
1,M,562.429453,NaN,335.0,NaN,0.0,NaN,0.654297,0.64827


### patients who stay in final list

In [35]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p3_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [36]:
df_waiting_time_fl.to_csv('results/decision_tree_p3_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",537.791,NaN,3.0,NaN,0.0,NaN,0.012,0.014
1,"Asian, Non-Hispanic",700.911,NaN,8.0,NaN,0.0,NaN,0.033,0.053
2,"Black, Non-Hispanic",541.791,NaN,97.0,NaN,0.0,NaN,0.394,0.325
3,Hispanic/Latino,676.099,NaN,44.0,NaN,0.0,NaN,0.179,0.160
4,"Multiracial, Non-Hispanic",247.441,NaN,2.0,NaN,0.0,NaN,0.008,0.005
5,"White, Non-Hispanic",686.003,NaN,92.0,NaN,0.0,NaN,0.374,0.439


In [37]:
df_gender_fl.to_csv('results/decision_tree_p3_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,563.186710,NaN,82.0,NaN,0.0,NaN,0.333333,0.35173
1,M,652.125321,NaN,164.0,NaN,0.0,NaN,0.666667,0.64827


In [38]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/decision_tree_p3_eth_prob.csv')
get_probability_by_ethinicy

<function simulation.get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl)>

# Random survival forest

In [39]:
events_per_replication_p1 = copy.deepcopy(events_per_replication)
events_per_replication_p2 = copy.deepcopy(events_per_replication)
events_per_replication_p3 = copy.deepcopy(events_per_replication)

## Policy 1

In [40]:
average_waiting_times, number_of_matches_list, results_p1_no, results_p1_ll, results_p1_fl = run_simulation(events_per_replication_p1,  
                                                                                                            replicates, random_forest, 
                                                                                                            policy='p1', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [11:28, 15.92time unit/s]                       


### recipients who receive a kidney

In [41]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p1_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [42]:
df_waiting_time_no.to_csv('results/random_forest_p1_eth_no.csv') 
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",533.733,NaN,13.0,NaN,2895.911,NaN,0.009,0.014
1,"Asian, Non-Hispanic",720.382,NaN,62.0,NaN,2923.898,NaN,0.045,0.053
2,"Black, Non-Hispanic",718.357,NaN,458.0,NaN,2962.810,NaN,0.332,0.325
3,Hispanic/Latino,748.942,NaN,219.0,NaN,2981.242,NaN,0.159,0.160
4,"Multiracial, Non-Hispanic",1102.513,NaN,6.0,NaN,3026.580,NaN,0.004,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1650.771,NaN,6.0,NaN,2915.049,NaN,0.004,0.004
6,"White, Non-Hispanic",593.724,NaN,617.0,NaN,2975.808,NaN,0.447,0.439


In [43]:
df_gender_no.to_csv('results/random_forest_p1_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,635.060411,NaN,513.0,NaN,2965.911841,NaN,0.37147,0.35173
1,M,693.190903,NaN,868.0,NaN,2971.195835,NaN,0.62853,0.64827


### patients who leave the waitlist

In [44]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p1_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [45]:
df_waiting_time_ll.to_csv('results/random_forest_p1_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1185.348,NaN,11.0,NaN,0.0,NaN,0.021,0.014
1,"Asian, Non-Hispanic",1352.222,NaN,27.0,NaN,0.0,NaN,0.051,0.053
2,"Black, Non-Hispanic",1601.877,NaN,181.0,NaN,0.0,NaN,0.343,0.325
3,Hispanic/Latino,1371.096,NaN,120.0,NaN,0.0,NaN,0.228,0.160
4,"Multiracial, Non-Hispanic",1819.000,NaN,2.0,NaN,0.0,NaN,0.004,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",700.181,NaN,2.0,NaN,0.0,NaN,0.004,0.004
6,"White, Non-Hispanic",1622.916,NaN,184.0,NaN,0.0,NaN,0.349,0.439


In [46]:
df_gender_ll.to_csv('results/random_forest_p1_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1605.062166,NaN,183.0,NaN,0.0,NaN,0.347249,0.35173
1,M,1494.036510,NaN,344.0,NaN,0.0,NaN,0.652751,0.64827


### patients who stay in final list

In [47]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p1_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [48]:
df_waiting_time_fl.to_csv('results/random_forest_p1_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",984.588,NaN,4.0,NaN,0.0,NaN,0.017,0.014
1,"Asian, Non-Hispanic",2420.620,NaN,11.0,NaN,0.0,NaN,0.046,0.053
2,"Black, Non-Hispanic",1565.041,NaN,88.0,NaN,0.0,NaN,0.368,0.325
3,Hispanic/Latino,1799.815,NaN,49.0,NaN,0.0,NaN,0.205,0.160
4,"Multiracial, Non-Hispanic",2087.359,NaN,4.0,NaN,0.0,NaN,0.017,0.005
5,"White, Non-Hispanic",1793.391,NaN,83.0,NaN,0.0,NaN,0.347,0.439


In [49]:
df_gender_fl.to_csv('results/random_forest_p1_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1896.212771,NaN,74.0,NaN,0.0,NaN,0.309623,0.35173
1,M,1656.732619,NaN,165.0,NaN,0.0,NaN,0.690377,0.64827


In [50]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/random_forest_p1_eth_prob.csv')

## Policy 2

In [51]:
average_waiting_times, number_of_matches_list, results_p2_no, results_p2_ll, results_p2_fl = run_simulation(events_per_replication_p2,  replicates, random_forest, policy='p2', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [01:32, 118.14time unit/s]                       


### recipients who receive a kidney

In [52]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p2_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [53]:
df_waiting_time_no.to_csv('results/random_forest_p2_eth_no.csv')
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1008.641,NaN,16.0,NaN,3022.626,NaN,0.012,0.014
1,"Asian, Non-Hispanic",1080.577,NaN,62.0,NaN,2963.084,NaN,0.045,0.053
2,"Black, Non-Hispanic",1120.023,NaN,456.0,NaN,2942.224,NaN,0.330,0.325
3,Hispanic/Latino,1059.806,NaN,244.0,NaN,2958.695,NaN,0.177,0.160
4,"Multiracial, Non-Hispanic",1089.325,NaN,9.0,NaN,3017.590,NaN,0.007,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1024.543,NaN,6.0,NaN,3006.033,NaN,0.004,0.004
6,"White, Non-Hispanic",1106.316,NaN,588.0,NaN,2972.769,NaN,0.426,0.439


In [54]:
df_gender_no.to_csv('results/random_forest_p2_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1118.686414,NaN,506.0,NaN,2962.463991,NaN,0.366401,0.35173
1,M,1088.990643,NaN,875.0,NaN,2959.800003,NaN,0.633599,0.64827


### patients who leave the waitlist

In [55]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p2_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [56]:
df_waiting_time_ll.to_csv('results/random_forest_p2_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",460.861,NaN,8.0,NaN,0.0,NaN,0.015,0.014
1,"Asian, Non-Hispanic",634.286,NaN,30.0,NaN,0.0,NaN,0.058,0.053
2,"Black, Non-Hispanic",523.739,NaN,166.0,NaN,0.0,NaN,0.321,0.325
3,Hispanic/Latino,525.302,NaN,93.0,NaN,0.0,NaN,0.180,0.160
4,"Multiracial, Non-Hispanic",238.575,NaN,1.0,NaN,0.0,NaN,0.002,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",301.240,NaN,1.0,NaN,0.0,NaN,0.002,0.004
6,"White, Non-Hispanic",575.954,NaN,218.0,NaN,0.0,NaN,0.422,0.439


In [57]:
df_gender_ll.to_csv('results/random_forest_p2_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,538.881565,NaN,179.0,NaN,0.0,NaN,0.346228,0.35173
1,M,556.648640,NaN,338.0,NaN,0.0,NaN,0.653772,0.64827


### patients who stay in final list

In [58]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p2_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [59]:
df_waiting_time_fl.to_csv('results/random_forest_p2_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",537.791,NaN,3.0,NaN,0.0,NaN,0.012,0.014
1,"Asian, Non-Hispanic",636.947,NaN,7.0,NaN,0.0,NaN,0.028,0.053
2,"Black, Non-Hispanic",547.189,NaN,95.0,NaN,0.0,NaN,0.386,0.325
3,Hispanic/Latino,675.587,NaN,45.0,NaN,0.0,NaN,0.183,0.160
4,"Multiracial, Non-Hispanic",247.441,NaN,2.0,NaN,0.0,NaN,0.008,0.005
5,"White, Non-Hispanic",670.977,NaN,94.0,NaN,0.0,NaN,0.382,0.439


In [60]:
df_gender_fl.to_csv('results/random_forest_p2_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,562.906555,NaN,84.0,NaN,0.0,NaN,0.341463,0.35173
1,M,646.536487,NaN,162.0,NaN,0.0,NaN,0.658537,0.64827


In [61]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/random_forest_p2_eth_prob.csv')

## Policy 3

### recipients who receive a kidney

In [62]:
average_waiting_times, number_of_matches_list, results_p3_no, results_p3_ll, results_p3_fl = run_simulation(events_per_replication_p3,  replicates, random_forest, policy='p3', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [21:09,  8.63time unit/s]                         


In [63]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p3_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [64]:
df_waiting_time_no.to_csv('results/random_forest_p3_eth_no.csv')
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1056.926,NaN,16.0,NaN,3014.436,NaN,0.012,0.014
1,"Asian, Non-Hispanic",1097.803,NaN,64.0,NaN,2957.480,NaN,0.046,0.053
2,"Black, Non-Hispanic",1130.518,NaN,454.0,NaN,2979.809,NaN,0.329,0.325
3,Hispanic/Latino,1066.911,NaN,242.0,NaN,2983.692,NaN,0.175,0.160
4,"Multiracial, Non-Hispanic",1152.263,NaN,9.0,NaN,2947.703,NaN,0.007,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1001.366,NaN,5.0,NaN,2897.080,NaN,0.004,0.004
6,"White, Non-Hispanic",1009.422,NaN,591.0,NaN,2965.052,NaN,0.428,0.439


In [65]:
df_gender_no.to_csv('results/random_forest_p3_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1070.593836,NaN,510.0,NaN,2975.077897,NaN,0.369298,0.35173
1,M,1061.493236,NaN,871.0,NaN,2971.833510,NaN,0.630702,0.64827


### patients who leave the waitlist

In [66]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p3_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [67]:
df_waiting_time_ll.to_csv('results/random_forest_p3_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",460.861,NaN,8.0,NaN,0.0,NaN,0.016,0.014
1,"Asian, Non-Hispanic",603.222,NaN,27.0,NaN,0.0,NaN,0.053,0.053
2,"Black, Non-Hispanic",543.445,NaN,170.0,NaN,0.0,NaN,0.332,0.325
3,Hispanic/Latino,574.308,NaN,96.0,NaN,0.0,NaN,0.188,0.160
4,"Multiracial, Non-Hispanic",238.575,NaN,1.0,NaN,0.0,NaN,0.002,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",700.181,NaN,2.0,NaN,0.0,NaN,0.004,0.004
6,"White, Non-Hispanic",570.249,NaN,208.0,NaN,0.0,NaN,0.406,0.439


In [68]:
df_gender_ll.to_csv('results/random_forest_p3_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,561.186147,NaN,177.0,NaN,0.0,NaN,0.345703,0.35173
1,M,562.429453,NaN,335.0,NaN,0.0,NaN,0.654297,0.64827


### patients who stay in final list

In [69]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p3_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [70]:
df_waiting_time_fl.to_csv('results/random_forest_p3_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",537.791,NaN,3.0,NaN,0.0,NaN,0.012,0.014
1,"Asian, Non-Hispanic",700.911,NaN,8.0,NaN,0.0,NaN,0.033,0.053
2,"Black, Non-Hispanic",541.791,NaN,97.0,NaN,0.0,NaN,0.394,0.325
3,Hispanic/Latino,676.099,NaN,44.0,NaN,0.0,NaN,0.179,0.160
4,"Multiracial, Non-Hispanic",247.441,NaN,2.0,NaN,0.0,NaN,0.008,0.005
5,"White, Non-Hispanic",686.003,NaN,92.0,NaN,0.0,NaN,0.374,0.439


In [71]:
df_gender_fl.to_csv('results/random_forest_p3_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,563.186710,NaN,82.0,NaN,0.0,NaN,0.333333,0.35173
1,M,652.125321,NaN,164.0,NaN,0.0,NaN,0.666667,0.64827


In [72]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/cox_prop_p1_eth_prob.csv')

# Cox proportional hazard models

In [73]:
events_per_replication_p1 = copy.deepcopy(events_per_replication)
events_per_replication_p2 = copy.deepcopy(events_per_replication)
events_per_replication_p3 = copy.deepcopy(events_per_replication)

## Policy 1

In [74]:
average_waiting_times, number_of_matches_list, results_p1_no, results_p1_ll, results_p1_fl = run_simulation(events_per_replication_p1,  
                                                                                                            replicates, cox_prop, 
                                                                                                            policy='p1', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [03:20, 54.59time unit/s]                        


### recipients who receive a kidney

In [75]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p1_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [76]:
df_waiting_time_no.to_csv('results/cox_prop_p1_eth_no.csv')
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",33.756,NaN,22.0,NaN,2581.999,NaN,0.016,0.014
1,"Asian, Non-Hispanic",192.454,NaN,60.0,NaN,2633.695,NaN,0.043,0.053
2,"Black, Non-Hispanic",182.022,NaN,495.0,NaN,2667.851,NaN,0.358,0.325
3,Hispanic/Latino,230.845,NaN,246.0,NaN,2615.168,NaN,0.178,0.160
4,"Multiracial, Non-Hispanic",632.089,NaN,9.0,NaN,2350.541,NaN,0.007,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",27.871,NaN,4.0,NaN,2488.246,NaN,0.003,0.004
6,"White, Non-Hispanic",295.788,NaN,545.0,NaN,2644.797,NaN,0.395,0.439


In [77]:
df_gender_no.to_csv('results/cox_prop_p1_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,187.407694,NaN,503.0,NaN,2667.309675,NaN,0.364229,0.35173
1,M,264.142756,NaN,878.0,NaN,2630.533727,NaN,0.635771,0.64827


### patients who leave the waitlist

In [78]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p1_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [79]:
df_waiting_time_ll.to_csv('results/cox_prop_p1_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1836.527,NaN,1.0,NaN,0.0,NaN,0.002,0.014
1,"Asian, Non-Hispanic",1839.805,NaN,28.0,NaN,0.0,NaN,0.054,0.053
2,"Black, Non-Hispanic",2539.390,NaN,136.0,NaN,0.0,NaN,0.263,0.325
3,Hispanic/Latino,1998.490,NaN,101.0,NaN,0.0,NaN,0.195,0.160
4,"Multiracial, Non-Hispanic",238.575,NaN,1.0,NaN,0.0,NaN,0.002,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",4635.292,NaN,4.0,NaN,0.0,NaN,0.008,0.004
6,"White, Non-Hispanic",2461.863,NaN,246.0,NaN,0.0,NaN,0.476,0.439


In [80]:
df_gender_ll.to_csv('results/cox_prop_p1_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,2514.451346,NaN,175.0,NaN,0.0,NaN,0.338491,0.35173
1,M,2295.101228,NaN,342.0,NaN,0.0,NaN,0.661509,0.64827


### patients who stay in final list

In [81]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p1_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [82]:
df_waiting_time_fl.to_csv('results/cox_prop_p1_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1096.555,NaN,2.0,NaN,0.0,NaN,0.008,0.014
1,"Asian, Non-Hispanic",3423.144,NaN,11.0,NaN,0.0,NaN,0.045,0.053
2,"Black, Non-Hispanic",2289.538,NaN,75.0,NaN,0.0,NaN,0.305,0.325
3,Hispanic/Latino,2647.926,NaN,43.0,NaN,0.0,NaN,0.175,0.160
4,"Multiracial, Non-Hispanic",1082.620,NaN,2.0,NaN,0.0,NaN,0.008,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",7039.137,NaN,1.0,NaN,0.0,NaN,0.004,0.004
6,"White, Non-Hispanic",3210.884,NaN,112.0,NaN,0.0,NaN,0.455,0.439


In [83]:
df_gender_fl.to_csv('results/cox_prop_p1_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,3429.608062,NaN,87.0,NaN,0.0,NaN,0.353659,0.35173
1,M,2489.757338,NaN,159.0,NaN,0.0,NaN,0.646341,0.64827


In [84]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/cox_prop_p1_eth_prob.csv')

## Policy 2

In [85]:
average_waiting_times, number_of_matches_list, results_p2_no, results_p2_ll, results_p2_fl = run_simulation(events_per_replication_p2,  
                                                                                                            replicates, cox_prop, 
                                                                                                            policy='p2', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [01:11, 154.12time unit/s]                       


### recipients who receive a kidney

In [86]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p2_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [87]:
df_waiting_time_no.to_csv('results/cox_prop_p2_eth_no.csv')
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",1008.641,NaN,16.0,NaN,2673.808,NaN,0.012,0.014
1,"Asian, Non-Hispanic",1080.577,NaN,62.0,NaN,2525.588,NaN,0.045,0.053
2,"Black, Non-Hispanic",1120.023,NaN,456.0,NaN,2571.281,NaN,0.330,0.325
3,Hispanic/Latino,1059.806,NaN,244.0,NaN,2541.112,NaN,0.177,0.160
4,"Multiracial, Non-Hispanic",1089.325,NaN,9.0,NaN,2663.454,NaN,0.007,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1024.543,NaN,6.0,NaN,2578.047,NaN,0.004,0.004
6,"White, Non-Hispanic",1106.316,NaN,588.0,NaN,2565.238,NaN,0.426,0.439


In [88]:
df_gender_no.to_csv('results/cox_prop_p2_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1118.686414,NaN,506.0,NaN,2582.319379,NaN,0.366401,0.35173
1,M,1088.990643,NaN,875.0,NaN,2552.055929,NaN,0.633599,0.64827


### patients who leave the waitlist

In [89]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p2_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [90]:
df_waiting_time_ll.to_csv('results/cox_prop_p2_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",460.861,NaN,8.0,NaN,0.0,NaN,0.015,0.014
1,"Asian, Non-Hispanic",634.286,NaN,30.0,NaN,0.0,NaN,0.058,0.053
2,"Black, Non-Hispanic",523.739,NaN,166.0,NaN,0.0,NaN,0.321,0.325
3,Hispanic/Latino,525.302,NaN,93.0,NaN,0.0,NaN,0.180,0.160
4,"Multiracial, Non-Hispanic",238.575,NaN,1.0,NaN,0.0,NaN,0.002,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",301.240,NaN,1.0,NaN,0.0,NaN,0.002,0.004
6,"White, Non-Hispanic",575.954,NaN,218.0,NaN,0.0,NaN,0.422,0.439


In [91]:
df_gender_ll.to_csv('results/cox_prop_p2_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,538.881565,NaN,179.0,NaN,0.0,NaN,0.346228,0.35173
1,M,556.648640,NaN,338.0,NaN,0.0,NaN,0.653772,0.64827


### patients who stay in final list

In [92]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p2_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [93]:
df_waiting_time_fl.to_csv('results/cox_prop_p2_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",537.791,NaN,3.0,NaN,0.0,NaN,0.012,0.014
1,"Asian, Non-Hispanic",636.947,NaN,7.0,NaN,0.0,NaN,0.028,0.053
2,"Black, Non-Hispanic",547.189,NaN,95.0,NaN,0.0,NaN,0.386,0.325
3,Hispanic/Latino,675.587,NaN,45.0,NaN,0.0,NaN,0.183,0.160
4,"Multiracial, Non-Hispanic",247.441,NaN,2.0,NaN,0.0,NaN,0.008,0.005
5,"White, Non-Hispanic",670.977,NaN,94.0,NaN,0.0,NaN,0.382,0.439


In [94]:
df_gender_fl.to_csv('results/cox_prop_p2_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,562.906555,NaN,84.0,NaN,0.0,NaN,0.341463,0.35173
1,M,646.536487,NaN,162.0,NaN,0.0,NaN,0.658537,0.64827


In [95]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/cox_prop_p2_eth_prob.csv')

## Policy 3

In [96]:
average_waiting_times, number_of_matches_list, results_p3_no, results_p3_ll, results_p3_fl = run_simulation(events_per_replication_p3,  
                                                                                                            replicates, cox_prop, 
                                                                                                            policy='p3', verbose=False)

Replicate # 0


Progress replication 0: 10959.15887291045time unit [10:39, 17.14time unit/s]                       


### recipients who receive a kidney

In [97]:
df_waiting_time_no, df_gender_no = get_summary_tables(results_p3_no, ethcat_mapping, real_ethcat_df, real_gender_df)

In [98]:
df_waiting_time_no.to_csv('results/cox_prop_p3_eth_no.csv')
df_waiting_time_no

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",542.443,NaN,14.0,NaN,2617.170,NaN,0.010,0.014
1,"Asian, Non-Hispanic",1289.101,NaN,61.0,NaN,2635.867,NaN,0.044,0.053
2,"Black, Non-Hispanic",882.187,NaN,484.0,NaN,2647.773,NaN,0.350,0.325
3,Hispanic/Latino,945.880,NaN,230.0,NaN,2603.857,NaN,0.167,0.160
4,"Multiracial, Non-Hispanic",1124.038,NaN,8.0,NaN,2419.228,NaN,0.006,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",2507.635,NaN,5.0,NaN,2343.550,NaN,0.004,0.004
6,"White, Non-Hispanic",1087.283,NaN,579.0,NaN,2616.406,NaN,0.419,0.439


In [99]:
df_gender_no.to_csv('results/cox_prop_p3_gender_no.csv')
df_gender_no

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,991.792138,NaN,524.0,NaN,2613.844441,NaN,0.379435,0.35173
1,M,1005.983994,NaN,857.0,NaN,2630.284303,NaN,0.620565,0.64827


### patients who leave the waitlist

In [100]:
df_waiting_time_ll, df_gender_ll = get_summary_tables(results_p3_ll, ethcat_mapping, real_ethcat_df, real_gender_df)

In [101]:
df_waiting_time_ll.to_csv('results/cox_prop_p3_eth_ll.csv')
df_waiting_time_ll

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",791.934,NaN,8.0,NaN,0.0,NaN,0.015,0.014
1,"Asian, Non-Hispanic",1011.621,NaN,32.0,NaN,0.0,NaN,0.061,0.053
2,"Black, Non-Hispanic",932.442,NaN,143.0,NaN,0.0,NaN,0.273,0.325
3,Hispanic/Latino,1009.284,NaN,109.0,NaN,0.0,NaN,0.208,0.160
4,"Multiracial, Non-Hispanic",238.575,NaN,1.0,NaN,0.0,NaN,0.002,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",999.230,NaN,3.0,NaN,0.0,NaN,0.006,0.004
6,"White, Non-Hispanic",946.996,NaN,228.0,NaN,0.0,NaN,0.435,0.439


In [102]:
df_gender_ll.to_csv('results/cox_prop_p3_gender_ll.csv')
df_gender_ll

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,967.022514,NaN,178.0,NaN,0.0,NaN,0.339695,0.35173
1,M,951.098058,NaN,346.0,NaN,0.0,NaN,0.660305,0.64827


### patients who stay in final list

In [103]:
df_waiting_time_fl, df_gender_fl = get_summary_tables(results_p3_fl, ethcat_mapping, real_ethcat_df, real_gender_df)

In [104]:
df_waiting_time_fl.to_csv('results/cox_prop_p3_eth_fl.csv')
df_waiting_time_fl

,ethcat,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_simu,sample_proportion
0,"Amer Ind/Alaska Native, Non-Hispanic",813.313,NaN,4.0,NaN,0.0,NaN,0.016,0.014
1,"Asian, Non-Hispanic",592.472,NaN,6.0,NaN,0.0,NaN,0.024,0.053
2,"Black, Non-Hispanic",858.777,NaN,93.0,NaN,0.0,NaN,0.378,0.325
3,Hispanic/Latino,1027.164,NaN,48.0,NaN,0.0,NaN,0.195,0.160
4,"Multiracial, Non-Hispanic",778.815,NaN,3.0,NaN,0.0,NaN,0.012,0.005
5,"Native Hawaiian/Other Pacific Islander, Non-Hi...",1064.119,NaN,1.0,NaN,0.0,NaN,0.004,0.004
6,"White, Non-Hispanic",1098.151,NaN,91.0,NaN,0.0,NaN,0.370,0.439


In [105]:
df_gender_fl.to_csv('results/cox_prop_p3_gender_fl.csv')
df_gender_fl

,gender,wt_mean,wt_std,count_mean,count_std,st_mean,st_std,prop_sim,sample_proportion
0,F,1018.870057,NaN,71.0,NaN,0.0,NaN,0.288618,0.35173
1,M,954.118834,NaN,175.0,NaN,0.0,NaN,0.711382,0.64827


In [106]:
get_probability_by_ethinicy(df_waiting_time_no, df_waiting_time_ll, df_waiting_time_fl).to_csv('results/cox_prop_p3_eth_prob.csv')